In [1]:
from preprocessing import get_data

DEFAULT_MODEL = "runs:/5c036be77ea045228b58b4fa52821f65/model"

sp500_data, last_day_df = get_data()

[*********************100%%**********************]  1 of 1 completed


/home/erik/DataspellProjects/Stock-Prediction/preprocessing.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data[f"SMA {time_period}"]      = ta.SMA(inputs, timeperiod = time_period)
/home/erik/DataspellProjects/Stock-Prediction/preprocessing.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data[f"EMA {time_period}"]      = ta.EMA(inputs, timeperiod = time_period)
/home/erik/DataspellProjects/Stock-Prediction/preprocessing.py:15: SettingWithCopyWarning: 
A value is trying to be set on a

In [2]:
last_day_df

,Close,Volume,Rendite,SMA 10,EMA 10,EMA 20,WMA 10,Momentum 10,SAR,RSI,ROC,%R,OBV,MACD,MACD_SIGNAL,MACD_HIST,CCI,ADOSC,%K,%D
Date,,,,,,,,,,,,,,,,,,,,
2023-11-16,2.927938,1.074054,0.070066,2.851697,2.853576,2.81436,2.874735,3.403774,2.790703,1.263104,1.251399,1.167591,2.140218,2.408888,0.898564,4.874446,0.950996,1.540242,0.886952,0.397163
